# Adapters

> Adapter implementations for integrating with plugin registries

In [ ]:
#| default_exp core.adapters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, List, Optional

from cjm_fasthtml_plugins.core.registry import UnifiedPluginRegistry
from cjm_fasthtml_workflow_transcription_single_file.core.protocols import PluginInfo, PluginRegistryProtocol

## PluginRegistryAdapter Class

Adapter that wraps a `UnifiedPluginRegistry` (from `cjm-fasthtml-plugins`) to implement `PluginRegistryProtocol`. This allows the workflow to work with any plugin registry that follows the unified plugin registry interface.

In [ ]:
#| export
class PluginRegistryAdapter:
    """Adapts app's UnifiedPluginRegistry to workflow's PluginRegistryProtocol.

    This adapter wraps the app's plugin registry and provides a simplified
    interface for the workflow to access transcription plugins.
    """

    def __init__(self, app_registry, category: str = "transcription"):
        """Initialize the adapter.

        Args:
            app_registry: The app's UnifiedPluginRegistry instance.
            category: Plugin category to filter by (default: "transcription").
        """
        self._registry = app_registry
        self._category = category

    def get_configured_plugins(self) -> List[PluginInfo]:
        """Get all configured transcription plugins (those with saved config files).

        Returns:
            List of PluginInfo for plugins that have saved configurations.
        """
        plugins = self._registry.get_plugins_by_category(self._category)
        return [
            PluginInfo(
                id=p.get_unique_id(),
                name=p.name,
                title=p.title,
                is_configured=p.is_configured,
                supports_streaming=self._check_streaming_support(p)
            )
            for p in plugins if p.is_configured
        ]

    def get_all_plugins(self) -> List[PluginInfo]:
        """Get all discovered transcription plugins (configured or not).

        Plugins without saved config files can still be used with their
        default configuration values from the schema.

        Returns:
            List of PluginInfo for all discovered plugins.
        """
        plugins = self._registry.get_plugins_by_category(self._category)
        return [
            PluginInfo(
                id=p.get_unique_id(),
                name=p.name,
                title=p.title,
                is_configured=p.is_configured,
                supports_streaming=self._check_streaming_support(p)
            )
            for p in plugins
        ]

    def get_plugin(self, plugin_id: str) -> Optional[PluginInfo]:
        """Get a specific plugin by ID.

        Args:
            plugin_id: Unique plugin identifier.

        Returns:
            PluginInfo if found, None otherwise.
        """
        plugin_meta = self._registry.get_plugin(plugin_id)
        if not plugin_meta:
            return None

        return PluginInfo(
            id=plugin_meta.get_unique_id(),
            name=plugin_meta.name,
            title=plugin_meta.title,
            is_configured=plugin_meta.is_configured,
            supports_streaming=self._check_streaming_support(plugin_meta)
        )

    def get_plugin_config(self, plugin_id: str) -> Dict[str, Any]:
        """Get the configuration for a plugin.

        Args:
            plugin_id: Unique plugin identifier.

        Returns:
            Configuration dictionary, empty dict if not configured.
        """
        return self._registry.load_plugin_config(plugin_id) or {}

    def _check_streaming_support(self, plugin_meta) -> bool:
        """Check if a plugin supports streaming output.

        Args:
            plugin_meta: Plugin metadata object.

        Returns:
            True if the plugin supports streaming.
        """
        if hasattr(plugin_meta, 'config_schema'):
            schema = plugin_meta.config_schema
            if isinstance(schema, dict):
                properties = schema.get("properties", {})
                return "stream_output" in properties or "streaming" in properties
        return False

## DefaultConfigPluginRegistryAdapter Class

Extended adapter that applies default configuration values when loading plugin configs. This ensures all required fields have values even if not explicitly configured.

In [ ]:
#| export
class DefaultConfigPluginRegistryAdapter(PluginRegistryAdapter):
    # COPY FROM: fasthtml_transcription/transcription/workflows/single_file/adapters.py
    # Lines 59-111: DefaultConfigPluginRegistryAdapter class
    #
    # Includes:
    # - Class docstring
    # - _apply_defaults(self, config: Dict[str, Any], schema: Optional[Dict[str, Any]]) method
    # - get_plugin_config(self, plugin_id: str) -> Dict[str, Any] method (overridden)
    """Plugin registry adapter that provides default config values for unconfigured plugins.

    This adapter wraps a UnifiedPluginRegistry and ensures that `load_plugin_config`
    returns default values from the plugin's schema when no saved config exists.
    This allows plugins to be used immediately without requiring a saved .json config file.
    """

    def __init__(self, registry: UnifiedPluginRegistry, category: str = "transcription"):
        """Initialize adapter with registry instance."""
        self._registry = registry
        self._category = category

    def get_plugins_by_category(self, category) -> list:
        """Get all plugins in a specific category."""
        return self._registry.get_plugins_by_category(category)

    def get_plugin(self, plugin_id: str):
        """Get a specific plugin by ID."""
        return self._registry.get_plugin(plugin_id)

    def load_plugin_config(self, plugin_id: str) -> Dict[str, Any]:
        """Load configuration for a plugin, using defaults if no saved config exists.

        Returns:
            Configuration dictionary. If no saved config exists, returns default
            values extracted from the plugin's config schema.
        """
        # First try to load saved config
        saved_config = self._registry.load_plugin_config(plugin_id)

        # If config exists and has values, return it
        if saved_config:
            return saved_config

        # No saved config - get default values from schema
        plugin_meta = self._registry.get_plugin(plugin_id)
        if plugin_meta and hasattr(plugin_meta, 'config_schema') and plugin_meta.config_schema:
            default_config = get_default_values_from_schema(plugin_meta.config_schema)
            return default_config

        # Fallback to empty dict
        return {}

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()